# Find sub-array in another array

based on this Stackoverflow question:
- https://stackoverflow.com/q/73522465/1609514

## Solution for 2D arrays

In [1]:
# %load find_array_in_array_2d.py
import numpy as np


def find_array_in_array_2d(ar, ar2):

    # Find all matches with first element of ar2
    match_idx = np.nonzero(ar[:-ar2.shape[0]+1, :-ar2.shape[1]+1] == ar2[0, 0])

    # Check remaining indices of ar2
    for i, j in list(np.ndindex(ar2.shape))[1:]:

        # End if no possible matches left
        if len(match_idx[0]) == 0:
            break

        # Index into ar offset by i, j
        nz2 = (match_idx[0] + i, match_idx[1] + j)

        # Find remaining matches with selected element
        to_keep = np.nonzero(ar[nz2] == ar2[i, j])[0]
        match_idx = match_idx[0][to_keep], match_idx[1][to_keep]

    return match_idx

In [2]:
# Example from stackoverflow question
ar = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 1, 2],
 [1, 1, 0, 1, 1, 0, 1, 1, 1, 1]]
)

ar2 = np.zeros((2,2))

In [3]:
match_idx = find_array_in_array_2d(ar, ar2)
match_idx

(array([4]), array([4]))

In [4]:
assert(match_idx == (np.array([4]), np.array([4])))

In [5]:
# Test with a random 2d array
shape = (10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j = np.random.randint(9, size=2)
    ar[i:i+2, j:j+2] = 10
ar2 = 10 * np.ones((2, 2)).astype(int)

ar, ar2

(array([[ 7,  6, 10, 10, 10, 10, 10, 10,  4,  9],
        [ 8, 10, 10, 10, 10, 10, 10, 10,  7,  1],
        [ 6, 10, 10,  5, 10, 10,  7,  1,  5,  9],
        [ 8,  1,  5,  9,  4,  0,  0,  7,  8,  3],
        [10, 10,  0,  5,  2,  3,  8,  5,  5,  3],
        [10, 10,  8,  6, 10, 10, 10, 10,  9,  4],
        [ 7,  4, 10, 10, 10, 10, 10, 10, 10,  1],
        [10, 10, 10, 10,  6,  4, 10, 10, 10,  3],
        [10, 10,  5,  4,  3,  3, 10, 10,  8,  9],
        [ 1,  5,  1,  9,  0,  0, 10, 10,  9,  5]]),
 array([[10, 10],
        [10, 10]]))

In [6]:
match_idx = find_array_in_array_2d(ar, ar2)
match_idx

(array([0, 0, 0, 0, 0, 1, 1, 4, 5, 5, 5, 6, 6, 6, 7, 7, 8]),
 array([2, 3, 4, 5, 6, 1, 4, 0, 4, 5, 6, 2, 6, 7, 0, 6, 6]))

In [7]:
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2], ar2) for i, j in zip(*match_idx)]
assert(all(checks))

## N-dimensional version

In [8]:
from functools import reduce

def find_array_in_array(ar, ar2):

    # Find all matches with first element of ar2
    slices = tuple(slice(None, -s + 1) for s in ar2.shape)
    match_idx = np.nonzero(ar[slices] == ar2.flat[0])
    
    # ar[:-ar2.shape[0]:, :-ar2.shape[1]]

    # Check remaining indices of ar2
    for idx in list(np.ndindex(ar2.shape))[1:]:

        if len(match_idx[0]) == 0:
            break

        # Index into ar offset by idx
        nz2 = tuple(m + i for m, i in zip(match_idx, idx))

        # Find remaining matches with selected element
        to_keep = np.nonzero(ar[nz2] == ar2[idx])[0]
        match_idx = tuple(m[to_keep] for m in match_idx)

    return match_idx

# 1d
assert(find_array_in_array(np.arange(0, 10), np.array([3, 4])) == (3,))
assert(find_array_in_array(np.arange(0, 10), np.array([4, -1]))[0].shape == (0,))

In [9]:
# Test with a random 2d array
shape = (10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j = np.random.randint(9, size=2)
    ar[i:i+2, j:j+2] = 10
ar2 = 10 * np.ones((2, 2)).astype(int)

ar, ar2

(array([[10, 10,  1,  7, 10, 10, 10,  4,  1,  0],
        [10, 10,  9,  2, 10, 10, 10,  8,  1,  0],
        [10, 10,  0,  9, 10, 10,  0,  5, 10, 10],
        [10, 10, 10, 10,  8,  9,  5, 10, 10, 10],
        [ 8,  2, 10, 10,  7,  2,  0, 10, 10,  7],
        [ 6,  0, 10, 10,  7,  2,  3, 10, 10,  3],
        [10, 10, 10, 10,  9,  0,  0,  6,  9,  6],
        [10, 10,  0,  2,  6,  9,  9,  9, 10, 10],
        [ 6,  0,  7,  6,  0, 10, 10,  5, 10, 10],
        [ 8,  0,  5,  1,  2, 10, 10,  0,  2,  6]]),
 array([[10, 10],
        [10, 10]]))

In [10]:
match_idx = find_array_in_array(ar, ar2)
match_idx

(array([0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 7, 8]),
 array([0, 4, 5, 0, 4, 0, 8, 2, 7, 2, 7, 2, 0, 8, 5]))

In [11]:
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2], ar2) for i, j in zip(*match_idx)]
assert(all(checks))

In [12]:
# Test with a random 3d array
shape = (10, 10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j, k = np.random.randint(10, size=3)
    ar[i:i+2, j:j+2, k:k+2] = 10
ar2 = 10 * np.ones((2, 2, 2)).astype(int)

In [13]:
match_idx = find_array_in_array(ar, ar2)
match_idx

(array([0, 1, 1, 1, 2, 3, 5, 6, 6, 6, 6, 8]),
 array([1, 0, 4, 6, 6, 5, 1, 3, 4, 7, 8, 2]),
 array([1, 5, 1, 6, 5, 4, 7, 5, 8, 1, 8, 0]))

In [14]:
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2, k:k+2], ar2) for i, j, k in zip(*match_idx)]
assert(all(checks))